In [1]:
import os
import sys
import blosc
import seaborn

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

sys.path.append("..")
sys.path.append("..\\..")

# from find_bounding_boxes import DetectionBatch
from task_03.my_batch import MnistBatch
from dataset import DatasetIndex, Dataset
from dataset import F, V, B
from task_03.my_batch import draw_stats, draw_digit
from detection_model import MyMnistModel

In [2]:
src = '..//task_03'

with open(os.path.join(src, 'mnist_pics.blk'), 'rb') as file:
    full_imgs = blosc.unpack_array(file.read())
    
with open(os.path.join(src, 'mnist_labels.blk'), 'rb') as file:
    full_labs = blosc.unpack_array(file.read())
    
src = (np.reshape(full_imgs, (65000, 28, 28)), full_labs)

In [3]:
LEN_MNIST = 65000
indy = DatasetIndex(np.arange(LEN_MNIST))

mnistset = Dataset(indy, batch_class=MnistBatch)
mnistset.cv_split([0.9, 0.1])


In [4]:
IMAGE_SIZE = 28
BATCH_SIZE = 100

In [5]:
placeholders_config = {
                       'images': {'shape': (None, None, 1),
                                 'data_format': 'channels_last',
                                 },
                       'labels': {'classes': 10,
                                 'name': 'targets'}
                       }

In [6]:
config={'inputs': placeholders_config,
        'loss': 'softmax_cross_entropy',
        'optimizer': {'name':'Adam', 'use_locking': True},
        'output': dict(ops=['labels', 'accuracy'])
       }

In [7]:
ppl = (mnistset.train.p
                     .load(src=src, fmt='ndarray')
                     .init_variable('acc_history', init_on_each_run=list)
                     .init_model('dynamic', MyMnistModel, 'MyMnistModel', config=config)
                     .train_model('MyMnistModel', fetches='accuracy', feed_dict={'images': B('images'),
                                                                                 'labels': B('labels'),
                                                                                 'keep_prob':0.9},
                                 save_to=V('acc_history'), mode='a'))

In [8]:
ppl.next_batch(BATCH_SIZE, n_epochs=None, shuffle=True)


start  [None, None, None, 1]
before  [None, None]


ValueError: The last dimension of the inputs to `Dense` should be defined. Found `None`.

In [ ]:
ppl_test_shift = (mnistset.test.p
                        .load(src=src, fmt='ndarray')
                        .enlarge_data() \
                        .find_bounding_box()
#                         .random_rotate(p=1, angle=(-10, 10), preserve_shape=True) \
#                         .random_scale(p=1, factor=(.8, 1.2)) \
                        .init_variable('augmented_acc_history', init_on_each_run=list)
                        .import_model('MyMnistModel', ppl)
                        .predict_model('MyMnistModel', fetches='accuracy', feed_dict={'input_images': B('images'),
                                                                      'input_labels': B('labels'),
                                                                      'keep_prob':1.0},
                        append_to=V('augmented_acc_history')))

In [ ]:
padded_pic = np.pad(pic, pad_width=[[left_pad, padding_size - left_pad], \
                                    [bottom_pad, padding_size - bottom_pad]], mode='constant')

In [ ]:
padded_pic